# Importing Required Libraries

We will use ImageDataGenerator for augmentation.

ResNet50 will be the model with some changes.




In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam, Adagrad, Adadelta
from tensorflow.keras.losses import BinaryCrossentropy, Hinge
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from google.colab import drive

drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading and normalizing the datasets


In [12]:
train_dir = r"/content/drive/MyDrive/Dataset/training_data"
test_dir = r"/content/drive/MyDrive/Dataset/testing_data"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.25
)

# Load the training data with the validation split
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(340, 340),
    batch_size=64,
    class_mode='binary',
    subset='training'  # Use the training subset
)

# Load the validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(340, 340),
    batch_size=64,
    class_mode='binary',
    subset='validation'  # Use the validation subset
)

# Loading the testing data
test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    )

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(340, 340),
    batch_size=64,
    class_mode='binary'
)

print(train_generator.class_indices)

Found 7128 images belonging to 2 classes.
Found 2375 images belonging to 2 classes.
Found 1425 images belonging to 2 classes.
{'correct': 0, 'incorrect': 1}


# Building the model

Utilizing ResNet50 and adding some layers within the model to enhance its performance

In [13]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(340, 340, 3)
    )

# Unfreeze the last 10 layers of the base model for fine-tuning
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Add custom layers on top
x = Flatten()(base_model.output)
x = Dropout(0.2)(x)  # Reducing dropout rate
x = Dense(512, activation='relu', kernel_regularizer=l2(0.005))(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.005))(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.005))(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer=Nadam(learning_rate=0.001),
    loss=BinaryCrossentropy(label_smoothing=0.1),
    metrics=['accuracy', 'AUC']
)

# Training the model

In [16]:
early_stopping_loss = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=False,
    verbose=1
)

early_stopping_acc = EarlyStopping(
    monitor='val_auc',
    patience=7,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=1,
    min_lr=1e-6
)

class_weight = {0: 1.7, 1: 1.0}
EPOCHS = 30
STEPS_PER_EPOCH = train_generator.samples // train_generator.batch_size
VALIDATION_STEPS = validation_generator.samples // validation_generator.batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS,
    callbacks=[early_stopping_loss, early_stopping_acc, lr_scheduler],
    class_weight=class_weight,
    verbose=1,
    epochs=EPOCHS
)

Epoch 1/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 300s 3s/step - AUC: 0.9964 - accuracy: 0.9795 - loss: 2.8015 - val_AUC: 0.6994 - val_accuracy: 0.5992 - val_loss: 2.1922 - learning_rate: 0.0010
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,accuracy,loss,val_AUC,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


  1/111 ━━━━━━━━━━━━━━━━━━━━ 1:17 704ms/step - AUC: 1.0000 - accuracy: 0.9844 - loss: 1.8103

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


111/111 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - AUC: 1.0000 - accuracy: 0.9844 - loss: 1.8103 - val_AUC: 0.9583 - val_accuracy: 0.4286 - val_loss: 2.2285 - learning_rate: 0.0010
Epoch 3/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 291s 2s/step - AUC: 0.9991 - accuracy: 0.9889 - loss: 1.6655 - val_AUC: 0.6830 - val_accuracy: 0.6242 - val_loss: 1.8131 - learning_rate: 0.0010
Epoch 4/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 521us/step - AUC: 1.0000 - accuracy: 0.9844 - loss: 1.4208 - val_AUC: 0.4000 - val_accuracy: 0.7143 - val_loss: 1.8169 - learning_rate: 0.0010
Epoch 5/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 292s 3s/step - AUC: 0.9983 - accuracy: 0.9804 - loss: 1.3958 - val_AUC: 0.4350 - val_accuracy: 0.4008 - val_loss: 1.7559 - learning_rate: 0.0010
Epoch 6/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 517us/step - AUC: 1.0000 - accuracy: 1.0000 - loss: 1.1937 - val_AUC: 0.5417 - val_accuracy: 0.5714 - val_loss: 1.6433 - learning_rate: 0.0010
Epoch 7/30
111/111 ━━━━━━━━━━━━━━━━━━━━ 292s 3s/step - AUC: 0.9984 - accuracy: 0.988

# Evaluate the model

In [17]:
results = model.evaluate(test_generator)
print(f'Test loss: {results[0]}')
print(f'Test accuracy: {results[1]}')
print(f'Test AUC: {results[2]}')

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - AUC: 0.9487 - accuracy: 0.9427 - loss: 0.8408
Test loss: 0.8594880104064941
Test accuracy: 0.934035062789917
Test AUC: 0.9408072829246521


# Save the model and its weights


In [18]:
# Save the model in Google Drive
model.save('/content/drive/MyDrive/project_prototype/model_prototype.h5')

# Save only the model weights
model.save_weights('/content/drive/MyDrive/project_prototype/model_weights.weights.h5')